In [60]:
import pandas as pd
import numpy as np
import openpyxl

Este codigo entrega un archivo Excel, como resultado de un csv procesado con este codigo. 
Este codigo contiene: 
- Llama a metodos de información para conocer el estado del DataFrame
- Funciones de limpieza
- Resultados

"Agradezco su consideración" Tsunami Martínez

### Desarrolla el codigo en Python para obtener un datafrae limpio con el csv proporcionado y regresalo en un archivo de excel

### Leer y conocer el archivo csv

In [61]:
# Leer archivo csv
df_data_set = pd.read_csv(
    "C:\\Users\\tsuda\\Downloads\\EXAMEN DATA ENGINEER JR\\EXAMEN DATA ENGINEER JR\\DATA SET  EJERCICIO.csv",
    delimiter=";",
)

In [62]:
df_data_set.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2,4,Credit Card,Takeaway,08/09/2023 00:00
1,TXN_4977031,Cake,4,3,12,Cash,In-store,16/05/2023
2,TXN_4271903,Cookie,4,1,ERROR,Credit Card,In-store,19/07/2023
3,TXN_7034554,Salad,2,5,10,UNKNOWN,UNKNOWN,27/04/2023
4,TXN_3160411,Coffee,2,2,4,Digital Wallet,In-store,11/06/2023


In [63]:
df_data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [64]:
df_data_set.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,12,7,8,19,5,5,381
top,TXN_9226047,Juice,5,3,6,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3021,159


### Limpiar DataFrame

In [ ]:
def clean_df(df):
    """
    Convierte las columnas de tipo object al tipo de dato adecuado para cada
    columna del DataFrame.
    """

    # Quantity a numeric
    df[["Quantity", "Price Per Unit", "Total Spent"]] = df[
        ["Quantity", "Price Per Unit", "Total Spent"]
    ].apply(pd.to_numeric, errors="coerce")

    # Transaction Date, reemplaza los valores a NaT
    df["Transaction Date"] = df["Transaction Date"].replace(
        {"UNKNOWN": pd.NaT, "ERROR": pd.NaT}
    )
    # Transaction Date a datetime
    df["Transaction Date"] = pd.to_datetime(
        df["Transaction Date"], format="mixed", errors="coerce"
    )
    # Transaction Date usa FFill para llenar fechas faltantes (NaT)
    df["Transaction Date"] = df["Transaction Date"].ffill()
    df["Transaction Date"] = df["Transaction Date"].dt.normalize()

    # Payment Method y Location reemplaza Unknown y Error con NA
    df[["Payment Method", "Location"]] = (
        df[["Payment Method", "Location"]]
        .replace({"UNKNOWN": pd.NA, "ERROR": pd.NA})
        .apply(lambda x: x.str.strip())
    )
    # Convierte a Nulos valores invalidos, elimina espacios
    df["Item"] = (
        df["Item"].replace({"": np.nan, "UNKNOWN": np.nan, "ERROR": np.nan}).str.strip()
    )
    # Solo toma en cuenta las filas donde existan mas de dos datos
    df = df[
        df[["Item", "Quantity", "Price Per Unit", "Total Spent"]].isna().sum(axis=1) < 3
    ]
    return df

In [66]:
# Aplicamos la funcion clean_df()
df = clean_df(df_data_set)

In [67]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-08-09
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,NaN,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,<NA>,<NA>,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-11-06


### Obtener Payment Method,	Location

In [68]:
# Verificar los valores NaN
df[['Location', 'Payment Method']].isna().sum()

Location          3958
Payment Method    3175
dtype: int64

In [69]:
# Calcular proporciones de valores conocidos en 'Location'
location_counts = df['Location'].value_counts(normalize=True) # Proporciones
location_known = location_counts.to_dict()
location_known

{'Takeaway': 0.5006626905235255, 'In-store': 0.4993373094764745}

In [70]:
#  Calcular proporciones de valores conocidos en 'Payment Method'
payment_counts = df['Payment Method'].value_counts(normalize=True)
payment_known = payment_counts.to_dict()
payment_known

{'Digital Wallet': 0.33582636750256634,
 'Credit Card': 0.3330400351957765,
 'Cash': 0.33113359730165715}

In [71]:
def fill_na_proportionally(column, known_proportions):
    """
    Esta funcion rellena proporcionalmete los valores faltantes
    usando una proporcion con los datos ya existentes donde no es NaN
    
    """
    # Filtra valores NaN y los cuenta
    nan_mask = column.isna() 
    num_missing = nan_mask.sum()  # Cantidad de valores Nulos
    
    # Si no hay Nulos, devolver la columna sin cambios
    
    if num_missing == 0:
        return column  
    
    # Asignar valores faltantes proporcionalmente usando np.random
    fill_values = np.random.choice(  
        list(known_proportions.keys()), 
        size=num_missing,  
        p=list(known_proportions.values()) 
    )
    
    # Rellenar los NaNs con los valores generados
    column.loc[nan_mask] = fill_values
    return column

In [72]:
# Aplicar la función a las columnas con valores faltantes
df.loc[:, "Location"] = fill_na_proportionally(df["Location"].copy(), location_known)
df.loc[:, "Payment Method"] = fill_na_proportionally(
    df["Payment Method"].copy(), payment_known
)

In [73]:
df[['Location', 'Payment Method']].isna().sum()

Location          0
Payment Method    0
dtype: int64

### Obtener Item,	Quantity,	Price Per Unit,	Total Spent	

In [74]:
df['Item'].isna().sum()

np.int64(963)

In [75]:
def dict_values(df, groupby_column, mean_column):
    """
    Recibe un DataFrame.
    Devuelve dos diccionarios:
    El primer grupando el "Item" y su precio
    El segundo por "Price Per Unit".
    """
    # Agrupa por valores de una columna
    # Obtiene de otra columna el promedio de cada una de ellas
    item_values = df.groupby(groupby_column)[mean_column].mean()
    item_price_dict = item_values.to_dict()  # Convierte a Diccionario
    
    # Crea a la inversa para que ahora el valor sea la clave del diccionario
    price_item_dict = {price: item for item, price in item_price_dict.items()}
    
    return item_price_dict, price_item_dict

In [76]:
# Obtiene los diccionarios con la funcion dict_values()
item_price_dict, price_item_dict = dict_values(df, 'Item', 'Price Per Unit')

In [77]:
item_price_dict, price_item_dict

({'Cake': 3.0,
  'Coffee': 2.0,
  'Cookie': 1.0,
  'Juice': 3.0,
  'Salad': 5.0,
  'Sandwich': 4.0,
  'Smoothie': 4.0,
  'Tea': 1.5},
 {3.0: 'Juice',
  2.0: 'Coffee',
  1.0: 'Cookie',
  5.0: 'Salad',
  4.0: 'Smoothie',
  1.5: 'Tea'})

In [78]:
def get_sells(df, item_dict, price_dict):
    """
    Rellena datos nulos o faltantes segun una serie de condiciones.
    Devuelve un DataFrame.
    """
    # Item y Price Per Unit son nulos 
    df.loc[df["Item"].isna() & df["Price Per Unit"].isna(), "Price Per Unit"] = (
        df["Total Spent"] / df["Quantity"]
    )

    # Item es nulo, Price Per Unit no lo es
    df.loc[df["Item"].isna() & df["Price Per Unit"].notna(), "Item"] = df[
        "Price Per Unit" # 3 y 4 es un valor repetido, por lo que diccionario no lo manejara
    ].map(lambda x: price_dict.get(x) if x != [3.0, 4.0] else None) 
    
    # Solucionar doble valor de 3
    item_values_three = df[df["Item"].isin(["Cake", "Juice"])]["Item"].value_counts(normalize=True)
    item_known = item_values_three.to_dict()
    
    # Usando la funcion fill_na_proportionally
    df.loc[:, "Item"] = fill_na_proportionally(
    df["Item"].copy(), item_known
    )
    
    # Doble valor de 4
    item_values_four= df[df["Item"].isin(["Sandwich", "Smoothie"])]["Item"].value_counts(normalize=True)
    item_known_ = item_values_four.to_dict()
    
    # Usando la funcion fill_na_proportionally
    df.loc[:, "Item"] = fill_na_proportionally(
    df["Item"].copy(), item_known_
    )
    
    # Item no es nulo, Price Per Unit no es nulo
    df.loc[df["Item"].notna() & df["Price Per Unit"].isna(), "Price Per Unit"] = df[
        "Item"
    ].map(item_dict)

    # Quantity no es nulo, Total Spent si es Nulo
    df.loc[df["Quantity"].notna() & df["Total Spent"].isna(), "Total Spent"] = (
        df["Quantity"] * df["Price Per Unit"]   
    )
    
    # Quantity es nulo, Total Spent no es nulo
    df.loc[df["Quantity"].isna() & df["Total Spent"].notna(), "Quantity"] = (
        df["Total Spent"] / df["Price Per Unit"]
    )
    return df

In [79]:
# Obtiene el DataFrame con los datos completos
df= get_sells(df, item_price_dict, price_item_dict)

In [80]:
df['Item'].isna().sum()

np.int64(0)

## Final

In [81]:
# Se obtiene el promedio de Quantity
quantity_mean = df['Quantity'].mean().round()

# Se agrega para rellenar filas, donde solo hay Quantity

df.loc[df["Quantity"].isna() & df["Total Spent"].isna(), "Quantity"] = df[
    "Quantity"
].fillna(quantity_mean)

In [82]:
# Se aplica la funcion get_datos()
df= get_sells(df, item_price_dict, price_item_dict)

In [83]:
df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

In [84]:
# Por la cantidad y tipo de datos que no se han podido obtener
# Aplique un dropna
df = df.dropna(subset=['Item'])

In [85]:
df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9994 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    9994 non-null   object        
 1   Item              9994 non-null   object        
 2   Quantity          9994 non-null   float64       
 3   Price Per Unit    9994 non-null   float64       
 4   Total Spent       9994 non-null   float64       
 5   Payment Method    9994 non-null   object        
 6   Location          9994 non-null   object        
 7   Transaction Date  9994 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 960.7+ KB


### Convertir a formato xlsx

In [88]:
df.to_excel('resultado_python.xlsx', index = False)